In [1]:
from keras.applications.resnet50 import ResNet50
from quiver_engine import server
from keras.models import Model

Using TensorFlow backend.


## Model setup

We use ResNet50 but any model from `keras.application` could be used.

In [2]:
resnet_model = ResNet50(
  include_top=False,
  weights='imagenet',
  input_shape=(224, 224, 3))

Select intermediate layer (so that the graph is not too big)

In [3]:
intermediate_layer = resnet_model.layers_by_depth[140][0]

intermediate_model = Model(inputs=[resnet_model.input], outputs=[intermediate_layer.output])

In [4]:
print("Number of layers in intermediate model:", len(intermediate_model.layers))
intermediate_model.summary()

Number of layers in intermediate model: 28
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________

Sanity check - this should be `(None, 224, 224, 3)`

In [5]:
intermediate_model.get_input_shape_at(0)

(None, 224, 224, 3)

Be sure you ran `make load_101_categories` before you run that to setup the images.

Also you could point *quiver* to other directory with images.

In [ ]:
imgs_path = "../../data/101_ObjectCategories/panda"

server.launch(intermediate_model, temp_folder="../../data/tmp", port=5001, input_folder=imgs_path)

Starting webserver from: /opt/anaconda3/envs/nnets/lib/python3.5/site-packages/quiver_engine


::ffff:127.0.0.1 - - [2018-02-03 11:55:48] "GET / HTTP/1.1" 200 824 0.007338
::ffff:127.0.0.1 - - [2018-02-03 11:55:48] "GET /app.c1d3d21cfba8c1aabed6806773c8f0ba.css HTTP/1.1" 200 5153 0.001998
::ffff:127.0.0.1 - - [2018-02-03 11:55:48] "GET /vendor.280c5ce1d78077ee8d66.js HTTP/1.1" 200 87579 0.002891
::ffff:127.0.0.1 - - [2018-02-03 11:55:48] "GET /app.14b8092d6ea379715007.js HTTP/1.1" 200 1179493 0.006635
::ffff:127.0.0.1 - - [2018-02-03 11:55:49] "GET /model HTTP/1.1" 200 25387 0.013865
::ffff:127.0.0.1 - - [2018-02-03 11:55:49] "GET /inputs HTTP/1.1" 200 975 0.001748
::ffff:127.0.0.1 - - [2018-02-03 11:55:49] "GET /favicon.ico HTTP/1.1" 200 16203 0.014770
::ffff:127.0.0.1 - - [2018-02-03 11:55:49] "GET /input-file/image_0008.jpg HTTP/1.1" 200 16657 0.012956
::ffff:127.0.0.1 - - [2018-02-03 11:55:49] "GET /input-file/image_0037.jpg HTTP/1.1" 200 10689 0.011262
::ffff:127.0.0.1 - - [2018-02-03 11:55:49] "GET /input-file/image_0032.jpg HTTP/1.1" 200 12029 0.004658
::ffff:127.0.0.1 - 